# Matching the WEST ICRH Antenna

In this notebook we investigate the various method to match a WEST ICRH antenna. By matching the antenna we mean to find 4 capacitances values $C_1$, $C_2$, $C_3$ and $C_4$ in order for the antenna to be operated at a given frequency $f_0$. 

In [1]:
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import skrf as rf
import numpy as np
import sys
from tqdm.notebook import tqdm

sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

In [4]:
rf.stylely()

C:\Users\JH218595\Documents\scikit-rf\skrf\plotting.py:1441: UserWarning: Style includes a parameter, 'interactive', that is not related to style.  Ignoring
  mpl.style.use(os.path.join(pwd, style_file))


In [5]:
from west_ic_antenna import WestIcrhAntenna

## Matching Each Sides Separately
Here, each side of the antenna is matched separatly, which leads to two set of capacitances $(C_1, C_2)$ and $(C_3,C_4)$.

In the following example, both sides of the antenna are matched at the frequency $f_0$, keeping opposite side unmatched (C=150pF):

In [6]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

In [7]:
C_match_left = ant.match_one_side(f_match=f0, side='left', solution_number=1)
C_match_right = ant.match_one_side(f_match=f0, side='right', solution_number=1)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

True solution #1: [50.75831692 48.74412321]
True solution #1: [50.56073699 48.93366323]
Left side matching point:  [50.75831692184981, 48.74412321496751, 150, 150]
Right side matching point:  [150, 150, 50.560736991732824, 48.933663234204275]


Let's have a look to the RF reflection coefficient of each sides:

In [8]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In reality, the precision at which one can tune the capacitance is not better than 1/100 pF, so one have to consider rounding optimal solutions to such precision :


In [9]:
C_match_left = ant.match_one_side(f_match=f0, side='left', 
                                  solution_number=1, decimals=2)
C_match_right = ant.match_one_side(f_match=f0, side='right', 
                                   solution_number=1, decimals=2)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [32.54974449 32.54974449]
True solution #1: [50.75831789 48.74412248]
Rounded result: [50.76, 48.74, 150.0, 150.0]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [50.56074253 48.93365805]
Rounded result: [150.0, 150.0, 50.56, 48.93]
Left side matching point:  [50.76, 48.74, 150.0, 150.0]
Right side matching point:  [150.0, 150.0, 50.56, 48.93]


Note that the performances are slightly degraded, but, it's real life! 

In [10]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))
ax.set_ylabel('$|S_{ii}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{11}$', '$S_{22}$'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Frequency Shift for Dipole Excitation
The coupling between antenna's sides requires shifting the frequency with respect to the matching frequency used for each side separately. Dipole excitation requires to shift the frequency to higher frequency to operate the antenna in a optimal conditions:

In [11]:
# dipole excitation
power = [1, 1]
phase = [0, rf.pi]
# combine both separate solutions
C_match = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_match)
# finding the optimum frequency
idx_f_opt = np.argmin(np.abs(s_act[:,0]))
f_opt = freq.f[idx_f_opt]
delta_f = f_opt - f0
print(f'Optimum frequency is f_opt={f_opt/1e6} MHz {delta_f/1e6} MHz shift' )

Optimum frequency is f_opt=55.212 MHz 0.212 MHz shift


In [12]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.axvline(f_opt/1e6, ls='--', color='k')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Matching Both Sides at the same time
It is also possible to optimize the antenna directly for the target frequency and for a given excitation. Note that it takes generally longer for the `scipy` optimizer to converge, especially on vacuum loading. A better algorithm could be employed.

In [90]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

# antenna excitation to match for
power = [1, 1]  # W
phase = [0, np.pi]  # dipole

ant.DEBUG=True
Cs = ant.match_both_sides(f_match=f0, power=power, phase=phase)

# Vacuum solutions obtained for dipole
#Cs = [52.08643927, 49.53448299, 50.62215495, 49.55183535]
# or 
# Cs = [51.77201166, 49.51663343, 50.70719552, 49.5951604 ]

Looking for individual solutions separately for 1st guess...
True solution #1: [50.75832176 48.74412023]
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [50.56074269 48.93365869]
Searching for the active match point solution...


C:\Users\JH218595\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:623: OptimizeWarning: Unknown solver options: rhoend
  return _minimize_cobyla(fun, x0, args, constraints, **options)


[50.75832176 48.74412023 50.56074269 48.93365869] 1.3576940626305094
[51.75832176 48.74412023 50.56074269 48.93365869] 1.274886982181167
[51.75832176 49.74412023 50.56074269 48.93365869] 1.2523642255780145
[51.75832176 49.74412023 51.56074269 48.93365869] 0.9055563286567979
[51.75832176 49.74412023 51.56074269 49.93365869] 1.3161261108390774
[51.91047087 49.78550337 52.19796492 48.17928085] 1.110476435691211
[51.83439631 49.7648118  51.8793538  48.55646977] 1.013719950068031
[51.99345932 49.80416324 51.50069968 48.93365869] 0.9422216192494802
[51.55322783 50.10654339 51.31820477 48.80038717] 0.8951869518209916
[51.55322783 49.9573847  51.16904609 48.66621053] 1.0680769001869896
[51.44265245 50.34320938 51.53461742 49.16771245] 1.1899570874056409
[51.57178183 50.16519458 51.48453242 48.62417452] 0.9800884286828089
[51.66400871 50.15987742 51.29835493 48.8110672 ] 0.8868354107725033
[51.71678793 50.04032831 51.09437268 48.7493128 ] 1.0085836027717778
[51.64501821 50.20988725 51.32189323 

In [91]:
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=Cs)

In [92]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [93]:
from IPython.core.display import HTML
def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter
    Pass the file path to the CSS file
    """
    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)

_set_css_style('custom.css')